In [581]:
import os, subprocess, sys
from pathlib import Path
import numpy as np
import collections
from pathlib import Path
import math

In [877]:
target = "pgh1"

In [878]:
flexi_path = f"/root/autodl-tmp/lbvs/test_dude5/{target}/docking_test/lib/"
rigid_path = f"/root/autodl-tmp/lbvs/test_dude5_rigid/{target}/docking_test/lib/"
flexi_rank_file = f"/root/autodl-tmp/lbvs/test_dude5/{target}/docking_test/ranked.txt"
rigid_rank_file = f"/root/autodl-tmp/lbvs/test_dude5_rigid/{target}/docking_test/ranked.txt"

In [879]:
with open(flexi_rank_file, "r") as tmp:
    flexi_rank = [a.strip() for a in tmp.readlines() if a != ""]
with open(rigid_rank_file, "r") as tmp:
    rigid_rank = [b.strip() for b in tmp.readlines() if b != ""]

In [880]:
rigid_lib_paths = [path.replace("./", rigid_path) 
                    for path in 
                       list(subprocess.check_output("find . -name '*best_position.pdbqt' -type f", shell=True, cwd=rigid_path).
                         decode(sys.stdout.encoding).split("\n"))][0:-1]
flexi_lib_paths = [path.replace("./", flexi_path) 
                    for path in 
                       list(subprocess.check_output("find . -name '*best_position.pdbqt' -type f", shell=True, cwd=flexi_path).
                         decode(sys.stdout.encoding).split("\n"))][0:-1]
ref_ind = [flexi_lib_paths.index(i) for i in flexi_lib_paths if ("_ligand" in Path(i).stem or "_lig" in Path(i).stem or "_lig1" in Path(i).stem or "_complex" in Path(i).stem)][0]
with open(flexi_lib_paths[ref_ind], "r") as tmp:
    lines = tmp.readlines()
    ref_torsion = int(lines[15].split()[1])
    ref_score_flexi = float(lines[3].replace("=", "").split()[-3])

with open(rigid_lib_paths[ref_ind], "r") as tmp:
    lines = tmp.readlines()
    ref_score_rigid = float(lines[3].replace("=", "").split()[-3])

del rigid_lib_paths[ref_ind]
del flexi_lib_paths[ref_ind]
ref_torsion

8

In [881]:
actives_ind = [flexi_lib_paths.index(i) for i in flexi_lib_paths if Path(i).stem.startswith("CHEMBL")]
actives_torsion = []
for ind in actives_ind:
    with open(flexi_lib_paths[ind], "r") as tmp:
        lines = tmp.readlines()
        actives_torsion.append(int(lines[15].split()[1]))
decoys_ind = [flexi_lib_paths.index(i) for i in flexi_lib_paths if not Path(i).stem.startswith("CHEMBL")]
decoys_torsion = []
for ind in decoys_ind:
    with open(flexi_lib_paths[ind], "r") as tmp:
        lines = tmp.readlines()
        decoys_torsion.append(int(lines[15].split()[1]))

In [882]:
flexi_scores = []
for lig in flexi_lib_paths:
    with open(lig, "r") as tmp:
        lines = tmp.readlines()
        flexi_scores.append(float(lines[3].replace("=", "").split()[-3]))
flexi_scores = np.array(flexi_scores)/ref_score_flexi

rigid_scores = []
for lig in rigid_lib_paths:
    with open(lig, "r") as tmp:
        lines = tmp.readlines()
        rigid_scores.append(float(lines[3].replace("=", "").split()[-3]))
rigid_scores = np.array(rigid_scores)/ref_score_rigid

In [883]:
lib_rbonds = []
lib_names = []
for lig in flexi_lib_paths:
    try:
        with open(lig, "r") as tmp:
            lines = tmp.readlines()
            lib_rbonds.append(int(lines[15].split()[1]))
            lib_names.append(Path(lig).stem.replace('_best_position', ''))
    except:
        pass

In [884]:
def torsion_from_pdbqt(lig):
    with open(f"{flexi_path}{lig}/{lig}_best_position.pdbqt", "r") as tmp:
        lines = tmp.readlines()
        return int(lines[15].split()[1])

def norm_score(score, ref):
    return -1/(1-score/ref)

In [885]:
# rigid_scores = normalize([rigid_scores])[0]
# flexi_scores = normalize([flexi_scores])[0]

In [886]:
import pandas as pd

In [887]:
sorted_ind = np.argsort((rigid_scores/flexi_scores)/(rigid_scores-flexi_scores)**2)
sorted_names = np.array(lib_names)[sorted_ind]
sorted_cr = ((rigid_scores/flexi_scores)/(rigid_scores-flexi_scores)**2)[sorted_ind]
sorted_torsion = [torsion_from_pdbqt(lig) for lig in sorted_names]

In [888]:
sorted_flexi_rank = [flexi_rank.index(name)+1 for name in sorted_names]
sorted_rigid_rank = [rigid_rank.index(name)+1 for name in sorted_names]

In [889]:
df = pd.DataFrame({"names":sorted_names,
                   "flexi_scores": flexi_scores[sorted_ind],
                   "rigid_scores": rigid_scores[sorted_ind],
                   "flexi_rank": sorted_flexi_rank,
                   "rigid_rank": sorted_rigid_rank,
                   "change rate":sorted_cr, 
                   "torsion":sorted_torsion})

In [894]:
np.mean((rigid_scores-flexi_scores)/flexi_scores)[actives_ind]

IndexError: invalid index to scalar variable.

In [891]:
np.mean((rigid_scores-flexi_scores)[actives_ind])

-0.0423185771611413

In [892]:
np.mean((rigid_scores/flexi_scores)[decoys_ind])

0.8461199382207486

In [893]:
np.mean((rigid_scores-flexi_scores)[decoys_ind])

-0.05848995730066564

In [875]:
np.mean((rigid_scores/flexi_scores)[actives_ind])/-np.mean((rigid_scores-flexi_scores)[actives_ind])

6.908222643766626

In [876]:
np.mean((rigid_scores/flexi_scores)[decoys_ind])/-np.mean((rigid_scores-flexi_scores)[decoys_ind])

11.404746886806548

In [845]:
import os
import subprocess
import numpy as np
import sys
from scipy.spatial import distance
def roc_auc_ef(total, actives, candidate_name):

    import os
    from importlib.resources import path
    from sklearn import metrics
    import matplotlib.pyplot as plt
    import numpy as np


    num_of_actives = len(actives)
    num_of_decoys = len(total) - num_of_actives

    act, dcy = 0, 0
    fpr, tpr = [], []
    for candidate in total:
        if candidate in actives:
            act += 1
        else:
            dcy += 1
        fpr.append(dcy/num_of_decoys)
        tpr.append(act/num_of_actives)
        
    return {candidate_name: round(metrics.auc(fpr, tpr), 15)}

In [846]:
roc_auc_ef(sorted_names,
           [name for name in sorted_names if name.startswith("CHEMBL")],
            target)

{'wee1': 0.559120521172638}

In [856]:
def roc_auc_ef(total, actives, candidate_name):

    import os
    from importlib.resources import path
    from sklearn import metrics
    import matplotlib.pyplot as plt
    import numpy as np


    num_of_actives = len(actives)
    num_of_decoys = len(total) - num_of_actives

    act, dcy = 0, 0
    fpr, tpr = [], []
    for candidate in total:
        if candidate in actives:
            act += 1
        else:
            dcy += 1
        fpr.append(dcy/num_of_decoys)
        tpr.append(act/num_of_actives)

    return [candidate_name, round(metrics.auc(fpr, tpr), 15)]

In [937]:
def loss_auc(target):

    def torsion_from_pdbqt(lig):
        with open(f"{flexi_path}{lig}/{lig}_best_position.pdbqt", "r") as tmp:
            lines = tmp.readlines()
            return int(lines[15].split()[1])
    from sklearn.preprocessing import normalize    
    import os
    import subprocess
    import numpy as np
    import sys
    from scipy.spatial import distance
    from pathlib import Path

    flexi_path = f"/root/autodl-tmp/lbvs/test_dude5/{target}/docking_test/lib/"
    rigid_path = f"/root/autodl-tmp/lbvs/test_dude5_rigid/{target}/docking_test/lib/"

    rigid_lib_paths = [path.replace("./", rigid_path) 
                    for path in 
                       list(subprocess.check_output("find . -name '*best_position.pdbqt' -type f", shell=True, cwd=rigid_path).
                         decode(sys.stdout.encoding).split("\n"))][0:-1]
    
    flexi_lib_paths = [path.replace("./", flexi_path) 
                        for path in 
                           list(subprocess.check_output("find . -name '*best_position.pdbqt' -type f", shell=True, cwd=flexi_path).
                             decode(sys.stdout.encoding).split("\n"))][0:-1]
    
    ref_ind = [flexi_lib_paths.index(i) for i in flexi_lib_paths if ("_ligand" in Path(i).stem or "_lig" in Path(i).stem or "_lig1" in Path(i).stem or "_complex" in Path(i).stem)][0]
    
    with open(flexi_lib_paths[ref_ind], "r") as tmp:
        lines = tmp.readlines()
        ref_torsion = int(lines[15].split()[1])
        ref_score_flexi = float(lines[3].replace("=", "").split()[-3])

    with open(rigid_lib_paths[ref_ind], "r") as tmp:
        lines = tmp.readlines()
        ref_score_rigid = float(lines[3].replace("=", "").split()[-3])

    del rigid_lib_paths[ref_ind]
    del flexi_lib_paths[ref_ind]
    
    actives_ind = [flexi_lib_paths.index(i) for i in flexi_lib_paths if Path(i).stem.startswith("CHEMBL")]
    actives_torsion = []
    for ind in actives_ind:
        with open(flexi_lib_paths[ind], "r") as tmp:
            lines = tmp.readlines()
            actives_torsion.append(int(lines[15].split()[1]))
            
    decoys_ind = [flexi_lib_paths.index(i) for i in flexi_lib_paths if not Path(i).stem.startswith("CHEMBL")]
    decoys_torsion = []
    for ind in decoys_ind:
        with open(flexi_lib_paths[ind], "r") as tmp:
            lines = tmp.readlines()
            decoys_torsion.append(int(lines[15].split()[1]))
    
    flexi_scores = []
    for lig in flexi_lib_paths:
        with open(lig, "r") as tmp:
            lines = tmp.readlines()
            flexi_scores.append(float(lines[3].replace("=", "").split()[-3]))
    flexi_scores = np.array(flexi_scores)

    rigid_scores = []
    for lig in rigid_lib_paths:
        with open(lig, "r") as tmp:
            lines = tmp.readlines()
            rigid_scores.append(float(lines[3].replace("=", "").split()[-3]))
    rigid_scores = np.array(rigid_scores)
    
    lib_rbonds = []
    lib_names = []
    for lig in flexi_lib_paths:
        try:
            with open(lig, "r") as tmp:
                lines = tmp.readlines()
                lib_rbonds.append(int(lines[15].split()[1]))
                lib_names.append(Path(lig).stem.replace('_best_position', ''))
        except:
            pass
    
    ranking = normalize([np.argsort(flexi_scores)+1])[0]
    # rigid_scores = normalize([rigid_scores])[0]
    # flexi_scores = normalize([flexi_scores])[0]
    sorted_ind = np.argsort(((flexi_scores-rigid_scores)/flexi_scores)/(ranking**4))[::-1]
    sorted_names = np.array(lib_names)[sorted_ind]
    # sorted_cr = ((rigid_scores/flexi_scores)/(rigid_scores-flexi_scores)**2)[sorted_ind]
    sorted_torsion = [torsion_from_pdbqt(lig) for lig in sorted_names]
    
    
    return roc_auc_ef(sorted_names, [name for name in sorted_names if name.startswith("CHEMBL")], target)

In [938]:
import os
targets = set(os.listdir("/root/autodl-tmp/lbvs/test_dude5")) - set(['ampc'])

In [939]:
na=[]
sc=[]
for tar in targets:
    a = loss_auc(tar)
    print(a)
    na.append(a[0])
    sc.append(a[-1])

['thb', 0.390188172043011]
['jak2', 0.591525423728814]
['lkha4', 0.703576769690927]
['aofb', 0.544565217391304]
['reni', 0.436546762589928]
['xiap', 0.640856031128405]
['hivpr', 0.411017218038751]
['aces', 0.633481594442326]
['glcm', 0.515039577836412]
['fgfr1', 0.533597049232908]
['fnta', 0.388159650932484]
['esr1', 0.512584704743466]
['ppard', 0.56611429350791]
['pnph', 0.359712230215827]
['src', 0.402082170369543]
['urok', 0.52782012195122]
['thrb', 0.39771150059714]
['ppara', 0.553879460856205]
['drd3', 0.496765657159659]
['pparg', 0.518613861386139]
['abl1', 0.377146699772398]
['mk14', 0.385985558702677]
['plk1', 0.446097201767305]
['wee1', 0.502931596091205]
['vgfr2', 0.340774167669475]
['comt', 0.544921875]
['def', 0.441652021089631]
['cp2c9', 0.513216845878136]
['bace1', 0.319295195954488]
['tysy', 0.413521545319465]
['fkb1a', 0.436050156739812]
['kpcb', 0.511551739399841]
['esr2', 0.506703556876996]
['hxk4', 0.653399668325041]
['lck', 0.525653739756772]
['pgh2', 0.470173639389

In [940]:
data = {"lig_names": na, "diff_score": sc}

In [941]:
import pandas as pd
pd.DataFrame.from_dict(data).to_csv("diff.csv")

In [942]:
import statistics
statistics.mean(sc)

0.5160246173552469